In [29]:
from scapy.all import *
import numpy as np

filter by:
- tcp packets
- egressing packets
- source ip

In [2]:
pcapfile = 'dataset/raw/attack_capture.pcap'

In [3]:
# store packets by src ip
packets = dict()

In [4]:
def store_pkt(pkt):
    global packets
    ip = pkt[IP].src
    if ip not in packets.keys():
        packets[ip] = []
    packets[ip].append(pkt)

In [5]:
# filter packets egressing from network
network = '10.0.0.64/26'

In [6]:
# filter syntax = https://biot.com/capstats/bpf.html
sniff(
    offline=pcapfile,
    store=False,
    count=100,
    filter="tcp and src net {net}".format(net=network),
    prn=store_pkt
)

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>

In [7]:
all_hosts = packets.keys()
all_hosts

dict_keys(['10.0.0.67'])

In [8]:
pkt = packets['10.0.0.67'][0]

In [9]:
pkt.show()

###[ Ethernet ]### 
  dst       = c2:02:64:e8:00:00
  src       = fe:f5:26:b1:c2:41
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 61713
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x3361
     src       = 10.0.0.67
     dst       = 10.2.2.5
     \options   \
###[ TCP ]### 
        sport     = 52806
        dport     = http
        seq       = 2010281419
        ack       = 0
        dataofs   = 10
        reserved  = 0
        flags     = S
        window    = 64240
        chksum    = 0x2d6c
        urgptr    = 0
        options   = [('MSS', 1460), ('SAckOK', b''), ('Timestamp', (3820972389, 0)), ('NOP', None), ('WScale', 7)]



In [10]:
float(pkt.time)

1624315639.847212

sort packets by timestamp

In [33]:
for host in packets.keys():
    packets[host] = sorted(packets[host], key=lambda pkt: pkt.time)

extract data from packets

for each source ip (host):
- get observation windows (sequential or sliding?)

In [11]:
T = 15 # observation window in seconds

In [44]:
packets['10.0.0.67'][0].time

Decimal('1624315639.847212')

In [43]:
start_time = np.floor(packets['10.0.0.67'][0].time)
start_time

1624315639

In [45]:
packets['10.0.0.67'][-1].time

Decimal('1624315647.331844')

In [46]:
end_time = np.ceil(packets['10.0.0.67'][-1].time)
end_time

1624315648

In [49]:
np.arange(start_time, end_time, 1)

array([1624315639, 1624315640, 1624315641, 1624315642, 1624315643,
       1624315644, 1624315645, 1624315646, 1624315647])

In [61]:
packets['10.0.0.67'][0].time < (1624315639 + 1)

True

In [74]:
packets['10.0.0.67'][0].time < (1624315639 + 1)

True

In [88]:
tmp = samples(packets['10.0.0.67'])

In [95]:
for k in tmp:
    print(len(k))

7
38
7
0
0
0
0
32
16


In [87]:
# arrange packets in 1 seconds intervals
def samples(packets, sample_interval=1):
    start_time = np.floor(packets[0].time)
    end_time = np.ceil(packets[-1].time)
    
    packet_samples = []
    
    num_packets = len(packets)
    packet_idx = 0
    for interval_start in np.arange(start_time, end_time, sample_interval):
        sample = []
        while packet_idx < num_packets and packets[packet_idx].time < int(interval_start + sample_interval):
            sample.append(packets[packet_idx])
            packet_idx +=1
        packet_samples.append(sample)
            
    return packet_samples

In [ ]:
def seqObsWindow(data,lengthObsWindow):
    nSamples,nMetrics=data.shape
    print("Observation window size: {}".format(lengthObsWindow))
    for s in np.arange(lengthObsWindow,nSamples,lengthObsWindow):
        print("\nAt sample: {}\n".format(s-1))
        subdata=data[s-lengthObsWindow:s,:]
        print(subdata)

In [ ]:
def slidingObsWindow(data,lengthObsWindow,slidingValue):
    nSamples,nMetrics=data.shape
    print("Observation window size: {}\nSliding value: {}".format(lengthObsWindow,slidingValue))
    for s in np.arange(lengthObsWindow,nSamples,slidingValue):
        print("\nAt sample: {}\n".format(s-1))
        subdata=data[s-lengthObsWindow:s,:]
        print(subdata)

for each observation window:
- Number of TCP packets
- Number of TCP packet *segments*
- Mean TCP packet length
- Variance TCP packet length
- shannon entropy of TCP destination IP
- shannon entropy of TCP destination port